In [11]:
import boto3
import botocore
import pandas as pd
from IPython.display import display, Markdown
import boto3
from botocore import UNSIGNED
from botocore.client import Config

## ПОДКЛЮЧАЮСЬ К AWS

In [12]:
s3 = boto3.client('s3',config=Config(signature_version=UNSIGNED))
s3_resource = boto3.resource('s3', config=Config(signature_version=UNSIGNED))

## ИЗУЧАЮ БАКЕТ


In [13]:
def list_bucket_contents(bucket, match='', size_mb=0):
    bucket_resource = s3_resource.Bucket(bucket)
    total_size_gb = 0
    total_files = 0
    match_size_gb = 0
    match_files = 0
    for key in bucket_resource.objects.all():
        key_size_mb = key.size/1024/1024
        total_size_gb += key_size_mb
        total_files += 1
        list_check = False
        if not match:
            list_check = True
        elif match in key.key:
            list_check = True
        if list_check and not size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')
        elif list_check and key_size_mb <= size_mb:
            match_files += 1
            match_size_gb += key_size_mb
            print(f'{key.key} ({key_size_mb:3.0f}MB)')

    if match:
        print(f'Размер файла {match_size_gb/1024:3.1f}GB with {match_files} files')            
    
    print(f'Размер корзиным {total_size_gb/1024:3.1f}GB with {total_files} files')
def csv_dataset(bucket, key,):
    data_source = {
            'Bucket': bucket,
            'Key': key
        }
    # Generate the URL to get Key from Bucket
    url = s3.generate_presigned_url(
        ClientMethod = 'get_object',
        Params = data_source
    )

    data = pd.read_csv(url)
    return data

In [14]:
list_bucket_contents(bucket='fisheries-catch-data',match='')

global-catch-data/ (  0MB)
global-catch-data/csv/ (  0MB)
global-catch-data/csv/rfmo_12.csv (194MB)
spatial-catch-maps/ (  0MB)
spatial-catch-maps/csv/ (  0MB)
Размер корзиным 0.2GB with 5 files


## Складываю В PANDAS

In [15]:
df = csv_dataset(bucket='fisheries-catch-data', key='global-catch-data/csv/rfmo_12.csv')

In [17]:
df.to_csv('fishes.csv')